In [1]:
import torch

In [2]:
dtype = torch.FloatTensor

batch_size, input_size, hidden_size, output_size = 64, 1000, 100, 10

x = torch.randn(batch_size, input_size).type(dtype)
y = torch.randn(batch_size, output_size).type(dtype)

w1 = torch.randn(input_size, hidden_size).type(dtype)
w2 = torch.randn(hidden_size, output_size).type(dtype)

learning_rate = 1e-6

In [3]:
print(x.shape, y.shape, w1.shape, w2.shape)

torch.Size([64, 1000]) torch.Size([64, 10]) torch.Size([1000, 100]) torch.Size([100, 10])


In [4]:
for i in range(500):
    # forward
    inh1 = x.mm(w1)
    outh1 = inh1.clamp(min=0) # relu activation function
    prediction = outh1.mm(w2)
    
    # loss
    loss = (prediction - y).pow(2).sum()
    print('i = %d, loss = %d' %(i, loss))
    
    # backward (WE NEED 'grad_loss_w1', 'grad_loss_w2' for weight update)
    grad_loss_prediction = 2.0 * (prediction - y)
    grad_prediction_w2 = outh1
    grad_loss_w2 = grad_prediction_w2.t().mm(grad_loss_prediction)
    
    grad_prediction_outh1 = w2
    grad_loss_outh1 = grad_loss_prediction.mm(grad_prediction_outh1.t())
    grad_loss_inh1 = grad_loss_outh1.clone()
    grad_loss_inh1[inh1<0] = 0
    grad_inh1_w1 = x
    grad_loss_w1 = grad_inh1_w1.t().mm(grad_loss_inh1)
    
    # weight update
    w1 -= learning_rate * grad_loss_w1
    w2 -= learning_rate * grad_loss_w2

i = 0, loss = 34095256
i = 1, loss = 30569718
i = 2, loss = 31038584
i = 3, loss = 30108498
i = 4, loss = 25207532
i = 5, loss = 17524738
i = 6, loss = 10374447
i = 7, loss = 5659890
i = 8, loss = 3123148
i = 9, loss = 1869985
i = 10, loss = 1248647
i = 11, loss = 918240
i = 12, loss = 722892
i = 13, loss = 593780
i = 14, loss = 500386
i = 15, loss = 428288
i = 16, loss = 370421
i = 17, loss = 322753
i = 18, loss = 282885
i = 19, loss = 249157
i = 20, loss = 220353
i = 21, loss = 195637
i = 22, loss = 174300
i = 23, loss = 155789
i = 24, loss = 139666
i = 25, loss = 125597
i = 26, loss = 113227
i = 27, loss = 102344
i = 28, loss = 92744
i = 29, loss = 84218
i = 30, loss = 76629
i = 31, loss = 69858
i = 32, loss = 63801
i = 33, loss = 58368
i = 34, loss = 53486
i = 35, loss = 49084
i = 36, loss = 45116
i = 37, loss = 41528
i = 38, loss = 38277
i = 39, loss = 35321
i = 40, loss = 32631
i = 41, loss = 30183
i = 42, loss = 27949
i = 43, loss = 25907
i = 44, loss = 24037
i = 45, loss = 2232

i = 481, loss = 0
i = 482, loss = 0
i = 483, loss = 0
i = 484, loss = 0
i = 485, loss = 0
i = 486, loss = 0
i = 487, loss = 0
i = 488, loss = 0
i = 489, loss = 0
i = 490, loss = 0
i = 491, loss = 0
i = 492, loss = 0
i = 493, loss = 0
i = 494, loss = 0
i = 495, loss = 0
i = 496, loss = 0
i = 497, loss = 0
i = 498, loss = 0
i = 499, loss = 0
